In [1]:
import numpy as np
import pandas as pd
from eunjeon import Mecab
import re
 
def cleanText(readData):
    text = re.sub('[-=+,#/\?:;^$.@*\"▼▲↑↓→←※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', str(readData))
    return text

In [2]:
data = pd.read_csv('raw_data_concat.csv')
data

,Column1,news_title
0,20010102,[코스닥] 작년 공모기업 평균 17% 하락 [표] 코스닥 기관 순매수.순매도 상위종...
1,20010103,"[코스닥] 쌍용정보통신 뉴브리지에 매각(5보) [뉴스] 인포뱅크,신한은행에 무선솔루..."
2,20010104,[B&B 브리프] “수익양호 동방등 10개사 유망” 外 [코스닥] 한국정보공학 “ ...
3,20010105,[거래소] 영업이익 상대우위 종목 6선-대우 운송 가전부품 올 영업이익 증가전망
4,20010108,"인성정보, “ERP사업서 올해 50억 매출가능” [B&B 브리프] 확정금리형 RP ..."
...,...,...
4448,20181221,"시너지이노베이션, 비씨엘바이오제약 지분 100% 보유딜로이트 안진, 국제조세 세미나..."
4449,20181224,“코스피 中 경기부양책 기대감에 반등 예상”연말 미국발 혼돈에 위험자산 기피…원/달...
4450,20181226,"[ET투자뉴스]영화테크, 업황은 개선 중… BUY(유지)-신..바이로메드, 3,70..."
4451,20181227,"<유>한올바이오파마, 전일 대비 7.15% 상승.. 일일회전율은..[한경로보뉴스] ..."


In [3]:
import konlpy

In [4]:
from konlpy.tag import Mecab

In [11]:
mecab = Mecab('C:\\mecab\\mecab-ko-dic')
data= pd.read_csv('raw_news_data_preprocessed_young.csv', encoding='CP949')
def analyze_morphs(df):
    df_size = len(df)
    for idx in np.arange(df_size):
        one_grams = []
        cleaned = cleanText(df.iloc[idx,1])
        parsed = mecab.pos(cleaned)
        for i in range(len(parsed)):
            word_and_wordclass = parsed[i][0]+'/'+parsed[i][1]
            one_grams.append(word_and_wordclass)
        df.loc[idx, 'analyzed_news_titles'] = ','.join(one_grams)
    return df

In [12]:
data_pos = analyze_morphs(data)
data_pos.head()

,date,news_title,analyzed_news_titles
0,20010102,[코스닥] 작년 공모기업 평균 17% 하락 [표] 코스닥 기관 순매수.순매도 상위종...,"코스닥/NNG,작년/NNG,공모/NNG,기업/NNG,평균/NNG,17/SN,하락/N..."
1,20010103,"[코스닥] 쌍용정보통신 뉴브리지에 매각(5보) [뉴스] 인포뱅크,신한은행에 무선솔루...","코스닥/NNG,쌍용/NNP,정보/NNG,통신/NNG,뉴/NNG,브리지/NNG,에/J..."
2,20010104,[B&B 브리프] “수익양호 동방등 10개사 유망” 外 [코스닥] 한국정보공학 “ ...,"BB/SL,브리프/NNG,“/SSO,수익/NNG,양호/NNG,동방/NNG,등/NNB..."
3,20010105,[거래소] 영업이익 상대우위 종목 6선-대우 운송 가전부품 올 영업이익 증가전망,"거래소/NNG,영업/NNG,이익/NNG,상대/NNG,우위/NNG,종목/NNG,6/S..."
4,20010108,"인성정보, “ERP사업서 올해 50억 매출가능” [B&B 브리프] 확정금리형 RP ...","인성/NNG,정보/NNG,“/SSO,ERP/SL,사업/NNG,서/JKB,올해/NNG..."


In [17]:
polar = pd.read_csv('polarity.csv')
polar.head()

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
1,가*/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
2,가*/JKS;있/VV;었/EP,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3,가*/VV,3,0.0,0.0,0.0,0.0,1.0,POS,1.0
4,가*/VV;ㄴ다*/EF,1,0.0,0.0,0.0,0.0,1.0,POS,1.0


In [49]:
import csv
with open('output.csv','w',newline='') as f:
    wr=csv.writer(f)
    wr.writerow(['Date','COMP','NEG','NEUT','None','POS'])
    for i in range(len(data_pos)):
        words = data_pos.iloc[i,2].split(',')
        sentiment = []
        count = 0
        for word in words:
            find = polar[polar['ngram']==word]
            if len(find)==0:
                continue
            find = np.array(find).tolist()
            sentiment.append(find[0][2:7])
            count+=1
        sentiment= (np.sum(sentiment, axis=0) / count).tolist()
        try:
            sentiment.insert(0,data_pos.iloc[i,0])
        except AttributeError:
            wr.writerow(list([data_pos.iloc[i,0],0,0,0,1,0]))
            print('데이터가 없음:' + str(data_pos.iloc[i,0]))
            continue


        wr.writerow(sentiment)
        if i%50==0:
            print('%d 번째 날짜 감정 되었습니다.' %i)


0 번째 날짜 감정 되었습니다.
50 번째 날짜 감정 되었습니다.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


데이터가 없음:20010427
100 번째 날짜 감정 되었습니다.
150 번째 날짜 감정 되었습니다.
데이터가 없음:20011016
200 번째 날짜 감정 되었습니다.
250 번째 날짜 감정 되었습니다.
300 번째 날짜 감정 되었습니다.
350 번째 날짜 감정 되었습니다.
400 번째 날짜 감정 되었습니다.
450 번째 날짜 감정 되었습니다.
500 번째 날짜 감정 되었습니다.
550 번째 날짜 감정 되었습니다.
600 번째 날짜 감정 되었습니다.
650 번째 날짜 감정 되었습니다.
700 번째 날짜 감정 되었습니다.
750 번째 날짜 감정 되었습니다.
800 번째 날짜 감정 되었습니다.
850 번째 날짜 감정 되었습니다.
900 번째 날짜 감정 되었습니다.
950 번째 날짜 감정 되었습니다.
1000 번째 날짜 감정 되었습니다.
1050 번째 날짜 감정 되었습니다.
1100 번째 날짜 감정 되었습니다.
1150 번째 날짜 감정 되었습니다.
1200 번째 날짜 감정 되었습니다.
1250 번째 날짜 감정 되었습니다.
1300 번째 날짜 감정 되었습니다.
1350 번째 날짜 감정 되었습니다.
1400 번째 날짜 감정 되었습니다.
1450 번째 날짜 감정 되었습니다.
1500 번째 날짜 감정 되었습니다.
1550 번째 날짜 감정 되었습니다.
1600 번째 날짜 감정 되었습니다.
1650 번째 날짜 감정 되었습니다.
1700 번째 날짜 감정 되었습니다.
1750 번째 날짜 감정 되었습니다.
1800 번째 날짜 감정 되었습니다.
1850 번째 날짜 감정 되었습니다.
1900 번째 날짜 감정 되었습니다.
1950 번째 날짜 감정 되었습니다.
2000 번째 날짜 감정 되었습니다.
2050 번째 날짜 감정 되었습니다.
2100 번째 날짜 감정 되었습니다.
2150 번째 날짜 감정 되었습니다.
2200 번째 날짜 감정 되었습니다.
2250 번째 날짜 감정 되었습니다.
2300 번째 날짜 감정 되었습니다.
2350 번째 날짜 감정 되었습니다.
2400 번째 날짜 감정 되었습니